<a href="https://colab.research.google.com/github/ekshaks/ekshaks.github.io/blob/master/vak/Vakyansh_Model2_(time_dict).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install git+https://github.com/huggingface/transformers.git datasets librosa torch==1.7.0 torchaudio==0.7.0 jiwer

In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

test_dataset = load_dataset("common_voice", "hi", split="test")
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model.to("cuda")

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

...

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [ ]:


resampler = torchaudio.transforms.Resample(48_000, 16_000)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
  batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

  with torch.no_grad():
      logits = model(inputs.input_values.to("cuda")).logits

      pred_ids = torch.argmax(logits, dim=-1)
      batch["pred_strings"] = processor.batch_decode(pred_ids, skip_special_tokens=True)
      return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

  0%|          | 0/127 [00:00<?, ?ex/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

WER: 32.946636


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import librosa
import soundfile as sf
import numpy as np

In [ ]:
counter

NameError: ignored

In [ ]:
b=[]
inputs = processor(a, sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values.to("cuda")).logits
    pred_ids = torch.argmax(logits, dim=-1)
    b = processor.batch_decode(pred_ids, skip_special_tokens=True)
    

In [ ]:
len(a[1])

36750

In [ ]:
a=[]
for i in range (1,42):
  path='/content/drive/MyDrive/'+str(i)+'.wav'
  speech_array, sampling_rate = torchaudio.load(path)
  x =speech_array[0].numpy()
  x=librosa.resample(np.asarray(x), sampling_rate, 16_000)

  a.append(x)


In [ ]:
processor1 = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-indian-english-enm-700")
model1 = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-indian-english-enm-700")
model1.to("cuda")

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [ ]:
pred_strings=[]
inputs = processor1(a, sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model1(inputs.input_values.to("cuda")).logits

    pred_ids_en = torch.argmax(logits, dim=-1)
    pred_strings= processor1.batch_decode(pred_ids_en, skip_special_tokens=True)


In [ ]:
" ".join(pred_strings)

'thank you so much mam this is now kah tei hain ki kob surat se eqpal ka kesa bhan jata hai khub surat se epal ka kisa bhan jata hai johni kab kan zindagika hisaban jata hai kuch khast log mirte hai zindagi me hch has log milte hai zindagi me jyn se kabhina bichar ne wala rish bhnjata hai adar abhinandan abhar namaskar adab sast a cal helo in a very god evening ladies in gentleman a very very warm welcome to this absolutely wonderful beautiful and ospicious day that brings together so many special people like you i am zuhi malhotra and today i have the pleasure of being a hosters for this day ustu aj ke is program ki shuruat may karna chahungi subsay pahley ap kapret se puna ha mi wagat karna chahungi vishwaka sabsi bata raj bhasha hindi tachniky samailen mai jahpar milker iscup srut suot sabko yah kahi mahab sabko hum celebrate kahrahehai mestumi shukriya adha karna chahungi abharvak karna c ongy sre suresh kumar jendal jika joki director hai residoke ar sat hi sat mashukriya karna ch

In [ ]:
t=0.02

In [ ]:
new=0
prev=0
x=pred_ids_en[22]
start=[]
end=[]
for i in range(len(x)):
  if x[i]!=0:
    if x[i]!=4:
      new=1
    else:
      new=2
  else:
    new=new
  if new!=prev:
    if new==2:
      end.append(i*t)
    elif new==1:
      start.append(i*t)
  prev=new

  
  


In [ ]:
def get_start_end(x):
    new=0
    prev=0
    x=pred_ids_en[x]
    start=[]
    end=[]
    for i in range(len(x)):
      if x[i]!=0:
        if x[i]!=4:
          new=1
        else:
          new=2
      else:
        new=new
      if new!=prev:
        if new==2:
          end.append(i*t)
        elif new==1:
          start.append(i*t)
      prev=new
    if len(start)!=len(end):
      end.append(5)
    return start,end




In [ ]:

def get_time_dict(x,k):
  
  words=pred_strings[x].split(' ')
  start,end=get_start_end(x)

  for i in range(0,len(words)):
    print('word :',words[i],'       Start_time:',(5*k+start[i]),'            end_time:',(5*k+end[i]))



In [ ]:
for r in range(len(pred_strings)):
  k=r
  get_time_dict(r,k)
  print(r)

word : thank        Start_time: 0.36             end_time: 0.5
word : you        Start_time: 0.54             end_time: 0.62
word : so        Start_time: 0.68             end_time: 0.78
word : much        Start_time: 0.8200000000000001             end_time: 0.96
word : mam        Start_time: 1.02             end_time: 1.22
word : this        Start_time: 1.78             end_time: 1.8800000000000001
word : is        Start_time: 1.92             end_time: 1.96
word : now        Start_time: 2.0             end_time: 2.12
word : kah        Start_time: 2.14             end_time: 2.2600000000000002
word : tei        Start_time: 2.3000000000000003             end_time: 2.38
word : hain        Start_time: 2.42             end_time: 2.52
word : ki        Start_time: 2.56             end_time: 2.68
word : kob        Start_time: 2.72             end_time: 2.9
word : surat        Start_time: 2.92             end_time: 3.2
word : se        Start_time: 3.24             end_time: 3.38
word : eqpal   